In [16]:
# Import libraries
import pandas as pd
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
from urllib.request import urlopen, Request
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [17]:
# Parameters 
n = 3
tickers = ['AAPL', 'TSLA', 'AMZN', 'MRNA', 'FB']

# Get Data
finwiz_url = 'https://finviz.com/quote.ashx?t='
news_tables = {}

In [18]:
for ticker in tickers:
    url = finwiz_url + ticker
    req = Request(url=url,headers={'user-agent': 'my-app/0.0.1'}) 
    resp = urlopen(req)    
    html = BeautifulSoup(resp, features="lxml")
    news_table = html.find(id='news-table')
    news_tables[ticker] = news_table

try:
    for ticker in tickers:
        df = news_tables[ticker]
        df_tr = df.findAll('tr')
    
        print ('\n')
        print ('Recent News Headlines for {}: '.format(ticker))
        
        for i, table_row in enumerate(df_tr):
            a_text = table_row.a.text
            td_text = table_row.td.text
            td_text = td_text.strip()
            print(a_text,'(',td_text,')')
            if i == n-1:
                break
except KeyError:
    pass



Recent News Headlines for AAPL: 
GlobalWafers Agrees to Buy Siltronic for About $4.5 Billion ( Dec-10-20 03:38AM )
Dow Jones Futures: Stock Market Rally Sell-Off Is Good News; DoorDash IPO Flies, Airbnb Prices High, This IPO Breaks Out ( Dec-09-20 10:23PM )
Facebook Risks Instagram-WhatsApp Breakup in Antitrust Case ( 07:23PM )


Recent News Headlines for TSLA: 
South Korea investigates fatal crash of Tesla Model X ( Dec-10-20 02:00AM )
Tesla Could Be Headed For $716 As Momentum Stocks Recover, Technical Analyst Says ( Dec-09-20 10:24PM )
Dow Jones Futures: Stock Market Rally Sell-Off Is Good News; DoorDash IPO Flies, Airbnb Prices High, This IPO Breaks Out ( 10:23PM )


Recent News Headlines for AMZN: 
Google, Amazon Fined $163 Million by French Data Watchdog ( Dec-10-20 03:32AM )
AWS and Arm Demonstrate Production-Scale Electronic Design Automation in the Cloud ( 03:01AM )
Facebook Risks Instagram-WhatsApp Breakup in Antitrust Case ( Dec-09-20 07:23PM )


Recent News Headlines for 

In [19]:
# Iterate through the news
parsed_news = []
for file_name, news_table in news_tables.items():
    for x in news_table.findAll('tr'):
        text = x.a.get_text() 
        date_scrape = x.td.text.split()

        if len(date_scrape) == 1:
            time = date_scrape[0]
            
        else:
            date = date_scrape[0]
            time = date_scrape[1]

        ticker = file_name.split('_')[0]
        
        parsed_news.append([ticker, date, time, text])

In [20]:
# Sentiment Analysis
analyzer = SentimentIntensityAnalyzer()

columns = ['Ticker', 'Date', 'Time', 'Headline']
news = pd.DataFrame(parsed_news, columns=columns)
scores = news['Headline'].apply(analyzer.polarity_scores).tolist()

df_scores = pd.DataFrame(scores)
news = news.join(df_scores, rsuffix='_right')


# View Data 
news['Date'] = pd.to_datetime(news.Date).dt.date

unique_ticker = news['Ticker'].unique().tolist()
news_dict = {name: news.loc[news['Ticker'] == name] for name in unique_ticker}

values = []
for ticker in tickers: 
    dataframe = news_dict[ticker]
    dataframe = dataframe.set_index('Ticker')
    dataframe = dataframe.drop(columns = ['Headline'])
    print ('\n')
    print (dataframe.head())
    
    mean = round(dataframe['compound'].mean(), 2)
    values.append(mean)
    
df = pd.DataFrame(list(zip(tickers, values)), columns =['Ticker', 'Mean Sentiment']) 
df = df.set_index('Ticker')
df = df.sort_values('Mean Sentiment', ascending=False)
print ('\n')
print (df)



              Date     Time    neg    neu    pos  compound
Ticker                                                    
AAPL    2020-12-10  03:38AM  0.000  0.816  0.184    0.2023
AAPL    2020-12-09  10:23PM  0.000  0.868  0.132    0.4404
AAPL    2020-12-09  07:23PM  0.259  0.741  0.000   -0.2732
AAPL    2020-12-09  05:55PM  0.000  0.661  0.339    0.4767
AAPL    2020-12-09  04:16PM  0.000  1.000  0.000    0.0000


              Date     Time    neg    neu    pos  compound
Ticker                                                    
TSLA    2020-12-10  02:00AM  0.508  0.492  0.000   -0.7351
TSLA    2020-12-09  10:24PM  0.000  1.000  0.000    0.0000
TSLA    2020-12-09  10:23PM  0.000  0.868  0.132    0.4404
TSLA    2020-12-09  10:10PM  0.000  1.000  0.000    0.0000
TSLA    2020-12-09  09:20PM  0.000  1.000  0.000    0.0000


              Date     Time    neg    neu    pos  compound
Ticker                                                    
AMZN    2020-12-10  03:32AM  0.000  0.825  0.175  